In [181]:
import pandas as pd
from geopy import distance

# ANALYSIS

In [182]:
"""
Travelling Salesman Problem has (n-1)! / 2 solutions. For 20 races, that means 6,08e+16
solutions.

First approach: clustering. Max number of points up to 6, and then minimize distances 
between
"""

'\nTravelling Salesman Problem has (n-1)! / 2 solutions. For 20 races, that means 6,08e+16\nsolutions.\n\nFirst approach: clustering. Max number of points up to 6, and then minimize distances \nbetween\n'

## The current calendar

In [183]:
circuits = pd.read_csv('../data/races_cleaned.csv', index_col = 0)

In [184]:
circuits.head()

,Date,Race,Circuit,Country,Latitude,Longitude
0,2020-03-08,Grand Prix of Qatar,Losail International Circuit,QATAR,25.491000,51.452068
1,2020-03-22,OR Thailand Grand Prix,Chang International Circuit,THAILAND,2.760191,101.736859
2,2020-04-05,Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES,30.138715,-97.636410
3,2020-04-19,Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA,-27.495926,-64.864078
4,2020-05-03,Gran Premio Red Bull de España,Circuito de Jerez,SPAIN,36.694447,-6.156317


In [185]:
# Now I can calculate the current distance:
current_distance = 0

for index in circuits.index[1:]:
    coord0 = (circuits.loc[index-1, 'Latitude'], circuits.loc[index-1, 'Longitude'])
    coord1 = (circuits.loc[index, 'Latitude'], circuits.loc[index, 'Longitude'])
    current_distance += round(distance.distance(coord0, coord1).km)

In [186]:
print('The current distance for the whole calendar is:', current_distance, 'km')

The current distance for the whole calendar is: 86324 km


#### Current distance is more than 86k km. The objective is minimize this distance

## First approach. Clustering Circuits

I will use Unsupervised Learning algorithms to cluster the circuits, and then optimizing the distance between clusters.

### Clustering with DBSCAN
The first clustering will be using DBSCAN, as I don't want to force any number of clusters.

In [187]:
# first of all I need to create a list of lists with the coordinates:
coordinates = []
for index in circuits.index:
    coord_list = [circuits.loc[index,'Latitude'], circuits.loc[index,'Longitude']]
    coordinates.append(coord_list)

In [188]:
# As the latitude goes from -90º to 90º and the longitude from -180º to 180º, first of
# all I will scale the coordinates to give them the same weight.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
coord_scale = scaler.fit_transform(coordinates)

In [189]:
coord_scale

array([[-0.24846584,  0.47539751],
       [-1.11898273,  1.33538315],
       [-0.07047329, -2.07435859],
       [-2.27769475, -1.51387629],
       [ 0.18059019, -0.50983846],
       [ 0.613838  , -0.4011866 ],
       [ 0.46034573, -0.21005766],
       [ 0.36728784, -0.3659332 ],
       [ 0.7182163 , -0.19119827],
       [ 0.80349503, -0.29290789],
       [ 1.10686416,  0.04829759],
       [ 0.65990494, -0.12320698],
       [ 0.58382058, -0.15211336],
       [ 0.76949383, -0.42189812],
       [ 0.45892218, -0.18762495],
       [ 0.3485146 , -0.40808144],
       [ 0.17443205,  1.9937042 ],
       [-2.69898859,  2.07936847],
       [-1.11898273,  1.33538315],
       [ 0.2878625 , -0.41525226]])

In [190]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5)
circuits_dbscan = dbscan.fit(coord_scale)
circuits['DBSCAN_Clusters'] = circuits_dbscan.labels_

In [191]:
circuits.groupby(['DBSCAN_Clusters']).size()

DBSCAN_Clusters
-1     7
 0    13
dtype: int64

#### There is a cluster with 13 items and 7 outliers

In [192]:
circuits.groupby(['DBSCAN_Clusters', 'Country']).size()

DBSCAN_Clusters  Country       
-1               ARGENTINA         1
                 AUSTRALIA         1
                 JAPAN             1
                 MALAYSIA          1
                 QATAR             1
                 THAILAND          1
                 UNITED STATES     1
 0               AUSTRIA           1
                 CZECH REPUBLIC    1
                 FINLAND           1
                 FRANCE            1
                 GERMANY           1
                 GREAT BRITAIN     1
                 ITALY             2
                 NETHERLANDS       1
                 SPAIN             4
dtype: int64

#### The distribuiton makes sense, as the outliers are far from each other while the items in the cluster are pretty close.

### Clustering with KMeans
In order to small calculations of distances, I will split the cluster with 13 items into several clusters of max 5 items each. This will simplify the optimizing calculation for the routes between the circuits into the cluster.

In [193]:
# creating the coordinates for circuits in cluster

cluster_coord = []
for index in circuits.query('DBSCAN_Clusters == 0').index:
    coord_list = [circuits.loc[index,'Latitude'], circuits.loc[index,'Longitude']]
    cluster_coord.append(coord_list)

In [194]:
cluster_coord

[[36.69444715, -6.15631689958845],
 [48.00734979999999, 0.1967379],
 [43.99938220000001, 11.3723647068196],
 [41.56946855, 2.25806310666666],
 [50.7328604, 12.4751047884026],
 [52.95964605, 6.5279741969904395],
 [60.88117595, 26.478826152075502],
 [49.2102429, 16.4506683],
 [47.223539200000005, 14.7604645],
 [52.071811600000004, -1.01429912415686],
 [43.9622107, 12.6840429850135],
 [41.07926445, -0.206414553372675],
 [39.4955257, -0.6257045]]

In [195]:
# in order to get a max of 5 items per cluster, I need 4 clusters
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)
circuits_clusters = kmeans.fit(cluster_coord)
circuits_clusters.labels_

array([2, 3, 0, 2, 0, 3, 1, 0, 0, 3, 0, 2, 2], dtype=int32)

In [196]:
# creating a new column to write the code of subcluster
circuits['Subcluster'] = 'None'

In [207]:
# creating a list with the index of the row and its subcluster code
sub_list = list(zip(circuits.query('DBSCAN_Clusters == 0').index, circuits_clusters.labels_))

In [208]:
# updating the subcluster code for the rows in cluster 0
for elem in sub_list:
    circuits.at[elem[0], 'Subcluster'] = elem[1]

In [209]:
circuits.query('DBSCAN_Clusters == 0')

,Date,Race,Circuit,Country,Latitude,Longitude,DBSCAN_Clusters,Subcluster
4,2020-05-03,Gran Premio Red Bull de España,Circuito de Jerez,SPAIN,36.694447,-6.156317,0,2
5,2020-05-17,SHARK Helmets Grand Prix de France,Le Mans,FRANCE,48.007350,0.196738,0,3
6,2020-05-31,Gran Premio d'Italia Oakley,Autodromo Internazionale del Mugello,ITALY,43.999382,11.372365,0,0
7,2020-06-07,Gran Premi Monster Energy de Catalunya,Circuit de Barcelona-Catalunya,SPAIN,41.569469,2.258063,0,2
8,2020-06-21,HJC Helmets Motorrad Grand Prix Deutschland,Sachsenring,GERMANY,50.732860,12.475105,0,0
9,2020-06-28,Motul TT Assen,TT Circuit Assen,NETHERLANDS,52.959646,6.527974,0,3
10,2020-07-12,Finland Grand Prix,KymiRing,FINLAND,60.881176,26.478826,0,1
11,2020-08-09,Monster Energy Grand Prix České republiky,Automotodrom Brno,CZECH REPUBLIC,49.210243,16.450668,0,0
12,2020-08-16,myWorld Motorrad Grand Prix von Österreich,Red Bull Ring - Spielberg,AUSTRIA,47.223539,14.760464,0,0
13,2020-08-30,British Grand Prix,Silverstone Circuit,GREAT BRITAIN,52.071812,-1.014299,0,3


In [311]:
circuits.groupby('Subcluster').count()['Circuit']

Subcluster
0       5
1       1
2       4
3       3
None    7
Name: Circuit, dtype: int64

#### I will start optimizing internal routes for subclusters 0 (5 circuits), 2 (4 circuits) and 3 (3 circuits).

## Optimizing routes

In [210]:
import mlrose

/usr/local/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



As the result of applying mlrose is in 'units' and the units of our coordinates are degrees of Latitude and Longitude, it is better to apply mlrose with a matrix of distances previously calculated in km with geopy.

### Subcluster 0

In [244]:
# creating a list with distances between circuits:
subcluster0 = circuits.query('Subcluster == 0').reset_index(drop=True)

dist_list = []

for i in subcluster0.index:
    for j in subcluster0.index:
        if i < j: 
            coord_i = (subcluster0.loc[i,'Latitude'],subcluster0.loc[i,'Longitude'])
            coord_j = (subcluster0.loc[j,'Latitude'],subcluster0.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list.append((i, j, dist))

In [245]:
dist_list

[(0, 1, 753.2093789744465),
 (0, 2, 697.4292716963196),
 (0, 3, 445.163498201435),
 (0, 4, 105.31854933139327),
 (1, 2, 331.6271130393199),
 (1, 3, 424.56157676462203),
 (1, 4, 752.9087233093135),
 (2, 3, 254.10488703569786),
 (2, 4, 650.6950903273602),
 (3, 4, 396.99937821188894)]

In [246]:
# Initialize fitness function object using coords_sub0
fitness_coords = mlrose.TravellingSales(distances = dist_list)

In [247]:
# Define optimization problem object
problem_fit = mlrose.TSPOpt(length = len(subcluster0), 
                            fitness_fn = fitness_coords, maximize = False)

In [248]:
# Solve using genetic algorithm - attempt 1
best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)

In [249]:
print(best_state)

[4 0 1 2 3]


In [250]:
print(best_fitness)

1841.2593065927463


In [251]:
# Solve using genetic algorithm - attempt 2
best_state, best_fitness = mlrose.genetic_alg(problem_fit, mutation_prob = 0.2, max_attempts = 100,
                                              random_state = 2)

In [252]:
print(best_state)

[4 0 1 2 3]


In [253]:
print(best_fitness)

1841.2593065927463


In [296]:
# creating the route with circuit names:
best_route0 = []
for i in best_state:
    name = subcluster0.loc[i, 'Circuit']
    best_route0.append(name)

In [297]:
best_route0

['Misano World Circuit Marco Simoncelli',
 'Autodromo Internazionale del Mugello',
 'Sachsenring',
 'Automotodrom Brno',
 'Red Bull Ring - Spielberg']

#### This is a round route. I don't want a round route, so I will subtract the distance from the last circuit to the first one.

In [298]:
# calculating and appending the total distance:
first = subcluster0.loc[subcluster0['Circuit'] == best_route0[0]].index[0]
last = subcluster0.loc[subcluster0['Circuit'] == best_route0[-1]].index[0]
last_stage = 0

for el in dist_list:
    if el[0] == first and el[1] == last:
        last_stage = el[2]
    elif el[0] == last and el[1] == first:
        last_stage = el[2]

route = best_fitness - last_stage

best_route0.append(round(route))

last_stage second step 396.99937821188894


In [299]:
best_route0

['Misano World Circuit Marco Simoncelli',
 'Autodromo Internazionale del Mugello',
 'Sachsenring',
 'Automotodrom Brno',
 'Red Bull Ring - Spielberg',
 1444]

### Subcluster 2

In [314]:
# creating a list with distances between circuits:
subcluster2 = circuits.query('Subcluster == 2').reset_index(drop=True)

dist_list2 = []

for i in subcluster2.index:
    for j in subcluster2.index:
        if i < j: 
            coord_i = (subcluster2.loc[i,'Latitude'],subcluster2.loc[i,'Longitude'])
            coord_j = (subcluster2.loc[j,'Latitude'],subcluster2.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list2.append((i, j, dist))

In [315]:
dist_list2

[(0, 1, 906.0416003864943),
 (0, 2, 709.20506184732),
 (0, 3, 576.0433647759187),
 (1, 2, 213.3824218268996),
 (1, 3, 335.7240942715326),
 (2, 3, 179.43584263435042)]

In [338]:
import math

def route_calc(dist_list):
    # Initialize fitness function object using coords_sub0
    fitness_coords = mlrose.TravellingSales(distances = dist_list)
    
    # Calculating the number of circuits depending on the number of distances in list
    # The num of distances is equal to n * (n-1) / 2, being n the number of circuits.
    # we want to find n having the num of distances, this is a quadratic function:
    # x**2 - x - 2y = 0, being x = n and y = num of distances.
    # Solving the equation with math library:
    
    a = 1
    b = -1
    c = -2*len(dist_list)
    # calculate the discriminant
    d = (b**2) - (4*a*c)
    # find two solutions
    sol1 = (-b-math.sqrt(d))/(2*a)
    sol2 = (-b+math.sqrt(d))/(2*a)
    
    # assigning the positive solution to length, needed for the algorithm
    length = max(sol1, sol2)
    
    # Define optimization problem object
    problem_fit = mlrose.TSPOpt(length = length, fitness_fn = fitness_coords, 
                                maximize = False)
    
    # Solve using genetic algorithm - attempt 1
    best_state1, best_fitness1 = mlrose.genetic_alg(problem_fit, random_state = 2)
    
    # Solve using genetic algorithm - attempt 2
    best_state2, best_fitness2 = mlrose.genetic_alg(problem_fit, mutation_prob = 0.2, 
                                                  max_attempts = 100, random_state = 2)
    
    if best_fitness1 < best_fitness2:
        return best_state1, round(best_fitness1)
    else:
        return best_state2, round(best_fitness2)

In [341]:
best_order2, round_dist2 = route_calc(dist_list2)

In [342]:
best_order2

array([3, 1, 2, 0])

In [343]:
# creating the route with circuit names:
best_route2 = []
for i in best_order2:
    name = subcluster2.loc[i, 'Circuit']
    best_route2.append(name)

In [344]:
best_route2

['Circuit Ricardo Tormo',
 'Circuit de Barcelona-Catalunya',
 'MotorLand Aragón',
 'Circuito de Jerez']

In [346]:
# calculating the no-round distance:
first = subcluster2.loc[subcluster2['Circuit'] == best_route2[0]].index[0]
last = subcluster2.loc[subcluster2['Circuit'] == best_route2[-1]].index[0]
last_stage = 0

for el in dist_list2:
    if el[0] == first and el[1] == last:
        last_stage = el[2]
    elif el[0] == last and el[1] == first:
        last_stage = el[2]

route = best_fitness - last_stage

best_route2.append(round(route))

In [347]:
best_route2

['Circuit Ricardo Tormo',
 'Circuit de Barcelona-Catalunya',
 'MotorLand Aragón',
 'Circuito de Jerez',
 1265]

### Subcluster 3

In [348]:
# creating a list with distances between circuits:
subcluster3 = circuits.query('Subcluster == 3').reset_index(drop=True)

dist_list3 = []

for i in subcluster3.index:
    for j in subcluster3.index:
        if i < j: 
            coord_i = (subcluster3.loc[i,'Latitude'],subcluster3.loc[i,'Longitude'])
            coord_j = (subcluster3.loc[j,'Latitude'],subcluster3.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list3.append((i, j, dist))

In [349]:
dist_list3

[(0, 1, 710.3446224068736),
 (0, 2, 460.3195563371291),
 (1, 2, 521.1997578348935)]

In [350]:
best_order3, round_dist3 = route_calc(dist_list3)

In [351]:
best_order3

array([2, 0, 1])

In [352]:
# creating the route with circuit names:
best_route3 = []
for i in best_order3:
    name = subcluster3.loc[i, 'Circuit']
    best_route3.append(name)

In [353]:
best_route3

['Silverstone Circuit', 'Le Mans', 'TT Circuit Assen']

In [354]:
# calculating the no-round distance:
first = subcluster3.loc[subcluster3['Circuit'] == best_route3[0]].index[0]
last = subcluster3.loc[subcluster3['Circuit'] == best_route3[-1]].index[0]
last_stage = 0

for el in dist_list3:
    if el[0] == first and el[1] == last:
        last_stage = el[2]
    elif el[0] == last and el[1] == first:
        last_stage = el[2]

route = best_fitness - last_stage

best_route3.append(round(route))

In [356]:
best_route3

['Silverstone Circuit', 'Le Mans', 'TT Circuit Assen', 1320]

### At this point, I have the best routes for the circuits inside the 3 subclusters. Still pending 1 circuit alone in the 4th subclusters and 7 circuits with no subcluster

### I will consider this 8 circuits as a new subcluster and calculate the best route between them

In [363]:
# creating the new dataframe
other_circuits = circuits.loc[(circuits['Subcluster'] == 1) | (circuits['Subcluster'] == 'None')].reset_index(drop=True)

In [364]:
other_circuits

,Date,Race,Circuit,Country,Latitude,Longitude,DBSCAN_Clusters,Subcluster
0,2020-03-08,Grand Prix of Qatar,Losail International Circuit,QATAR,25.491000,51.452068,-1,None
1,2020-03-22,OR Thailand Grand Prix,Chang International Circuit,THAILAND,2.760191,101.736859,-1,None
2,2020-04-05,Red Bull Grand Prix of The Americas,Circuit Of The Americas,UNITED STATES,30.138715,-97.636410,-1,None
3,2020-04-19,Gran Premio Motul de la República Argentina,Termas de Río Hondo,ARGENTINA,-27.495926,-64.864078,-1,None
4,2020-07-12,Finland Grand Prix,KymiRing,FINLAND,60.881176,26.478826,0,1
5,2020-10-18,Motul Grand Prix of Japan,Twin Ring Motegi,JAPAN,36.533647,140.229985,-1,None
6,2020-10-25,Australian Motorcycle Grand Prix,Phillip Island,AUSTRALIA,-38.496688,145.238917,-1,None
7,2020-11-01,Shell Malaysia Motorcycle Grand Prix,Sepang International Circuit,MALAYSIA,2.760191,101.736859,-1,None


In [365]:
dist_list_other = []

for i in other_circuits.index:
    for j in other_circuits.index:
        if i < j: 
            coord_i = (other_circuits.loc[i,'Latitude'],other_circuits.loc[i,'Longitude'])
            coord_j = (other_circuits.loc[j,'Latitude'],other_circuits.loc[j,'Longitude'])

            dist = distance.distance(coord_i, coord_j).km

            dist_list_other.append((i, j, dist))

In [366]:
dist_list_other

[(0, 1, 5934.373927069669),
 (0, 2, 13027.280851968753),
 (0, 3, 13748.464271442162),
 (0, 4, 4365.819538710464),
 (0, 5, 8270.183074011635),
 (0, 6, 12038.62218753436),
 (0, 7, 5934.373927069669),
 (1, 2, 15827.81888198896),
 (1, 3, 16923.729646057596),
 (1, 4, 8943.668223441835),
 (1, 5, 5435.523367568314),
 (1, 6, 6386.1405068187905),
 (1, 7, 0.0),
 (2, 3, 7269.066592961215),
 (2, 4, 8726.736191782999),
 (2, 5, 10479.282482273198),
 (2, 6, 14274.208157616871),
 (2, 7, 15827.81888198896),
 (3, 4, 12706.666915492706),
 (3, 5, 17459.966421628003),
 (3, 6, 12058.974243919296),
 (3, 7, 16923.729646057596),
 (4, 5, 7663.212564868035),
 (4, 6, 15179.42626628677),
 (4, 7, 8943.668223441835),
 (5, 6, 8323.338197187517),
 (5, 7, 5435.523367568314),
 (6, 7, 6386.1405068187905)]